# **電玩遊戲資料分析與預測**

這是KAGGLE引進data的指令

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
import seaborn as sns # Matplot的基礎上實作更高階的視覺化API
import matplotlib.pyplot as plt

import operator
import pandas
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('../input/vgsales.csv')
df.plot(kind = 'scatter' ,
       x = "Year" ,
       y = "Global_Sales" ,figsize=(20,20))

按照發行日排序

In [ ]:
df.sort_values(by=['Year']).head()

查看最大出版西元年數

In [ ]:
year_data = df['Year']
print("Max Year Value: ", year_data.max())

發現有不對的出版年數，查看相對應的那筆資料

In [ ]:
max_entry = year_data.idxmax()
max_entry = df.iloc[max_entry]
pd.DataFrame(max_entry).T

經上網查證後，將該遊戲出版年數改成正確年數

In [ ]:
df['Year'] = df['Year'].replace(2020.0, 2009.0)
df.loc[[5957]]

查看總數

In [ ]:
#df.info()
print("Number of games: ", len(df))
publishers = df['Publisher'].unique()
print("Number of publishers: ", len(publishers))
platforms = df['Platform'].unique()
print("Number of platforms: ", len(platforms))
genres = df['Genre'].unique()
print("Number of genres: ", len(genres))

查看是否有null值，若有就丟棄

In [ ]:
print(df.isnull().sum())
df = df.dropna()

# **總出版遊戲數量 - 長條圖**

In [ ]:
#數值資料之統計值
#df.describe()
#遊戲平台 數量統計
#df['Platform'].value_counts()
#遊戲類型 數量統計
#df['Genre'].value_counts()
#遊戲出版商 數量統計
#df['Publisher'].value_counts()

利用seaborn套件畫出，總出版遊戲數量之遊戲類型。其中動作類遊戲居冠，運動類遊戲其次

In [ ]:
Genre = pd.crosstab(df.Genre,df.Genre)
GenreTotal = Genre.sum(axis=1).sort_values(ascending = False)

plt.figure(figsize=(20,5))
ax = sns.barplot(y = GenreTotal.index, x = GenreTotal.values, orient='h')
ax.set_xlabel(xlabel='The amount of games', fontsize=20)
ax.set_ylabel(ylabel='Genre', fontsize=20)
ax.set_title(label='Total Genre Games Released', fontsize=30)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=20)
plt.show()

總出版遊戲數量之遊戲平台。可以看出DS和PS2兩平台的銷售量最多

In [ ]:
Platform = pd.crosstab(df.Platform,df.Genre)
PlatformTotal = Platform.sum(axis=1).sort_values(ascending = False)

plt.figure(figsize=(20,10))
ax = sns.barplot(y = PlatformTotal.index, x = PlatformTotal.values, orient='h')
ax.set_xlabel(xlabel='The amount of games', fontsize=20)
ax.set_ylabel(ylabel='Platform', fontsize=20)
ax.set_title(label='Total Platform Games Released', fontsize=30)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=20)
plt.show()

總出版遊戲數量之出版商。由於出版商眾多，這裡只列出前十名。前十名中，美國公司有4家，日本公司有5家，法國公司有1家

In [ ]:
Publisher = pd.crosstab(df.Publisher,df.Genre)
PublisherTotal = Publisher.sum(axis=1).sort_values(ascending = False)[:10]

plt.figure(figsize=(15,5))
colors = sns.color_palette("bright")
ax = sns.barplot(y = PublisherTotal.index, x = PublisherTotal.values, orient='h', palette=colors)
ax.set_xlabel(xlabel='The amount of games', fontsize=20)
ax.set_ylabel(ylabel='Publisher', fontsize=20)
ax.set_title(label='Top 10 Total Publisher Games Released', fontsize=30)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=20)
plt.show()

In [ ]:
y = df.groupby(['Year']).sum()
y = y['Global_Sales']
x = y.index.astype(int)

plt.figure(figsize=(12,8))
ax = sns.barplot(y = y, x = x)
ax.set_xlabel(xlabel='Millions', fontsize=15)
ax.set_xticklabels(labels = x, fontsize=12, rotation=50)
ax.set_ylabel(ylabel='Year', fontsize=15)
ax.set_title(label='Game Sales:Millions Per Year', fontsize=20)
plt.show();

# **歷年出版遊戲數量 - 長條圖 + 熱力圖**

歷年出版遊戲數量

In [ ]:
Publisher = pd.crosstab(df.Year,df.Genre)
PublisherTotal = Publisher.sum(axis=1)
x = PublisherTotal.values
y = PublisherTotal.index.astype(int)

plt.figure(figsize=(15,10))
colors = sns.color_palette("bright")
ax = sns.barplot(y = y, x = x, orient='h', palette=colors)
ax.set_xlabel(xlabel='Number of releases', fontsize=20)
ax.set_ylabel(ylabel='Year', fontsize=20)
ax.set_title(label='Game Releases Per Year', fontsize=30)
ax.set_yticklabels(labels = y, fontsize=18)
plt.show()

利用seaborn套件畫出歷年出版遊戲數類型熱力圖

In [ ]:
table_countbyGenre = pd.pivot_table(df,values=['Global_Sales'],index=['Year'],columns=['Genre'],aggfunc='count',margins=False)
y = table_countbyGenre.index.astype(int)

plt.figure(figsize=(15,15))
ax = sns.heatmap(table_countbyGenre['Global_Sales'],linewidths=.5,annot=True,fmt='2.0f',vmin=0,cmap='coolwarm')
ax.set_xlabel(xlabel='Genre', fontsize=20)
ax.set_ylabel(ylabel='Year', fontsize=20)
ax.set_title(label='Count of games by Genre', fontsize=30)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=20)
ax.set_yticklabels(labels = y, fontsize=20)
plt.show()

歷年遊戲平台出版遊戲數熱力圖

In [ ]:
most_pub = df.groupby('Platform').Global_Sales.sum()
most_pub.sort_values(ascending=False)
table_countbyPlatform = pd.pivot_table(df[df.Platform.isin(most_pub.sort_values(ascending=False).index)],values=['Global_Sales'],index=['Year'],columns=['Platform'],aggfunc='count',margins=False)
y = table_countbyPlatform.index.astype(int)

plt.figure(figsize=(25,15))
ax = sns.heatmap(table_countbyPlatform['Global_Sales'],linewidths=.5,annot=True,fmt='2.0f',vmin=0,cmap='coolwarm')
ax.set_xlabel(xlabel='Platform', fontsize=20)
ax.set_ylabel(ylabel='Year', fontsize=20)
ax.set_title(label='Count of games by Platform', fontsize=30)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=20)
ax.set_yticklabels(labels = y, fontsize=20)
plt.show()

歷年遊戲公司出版遊戲數熱力圖。這裡只列出排行前20名

In [ ]:
most_pub = df.groupby('Publisher').Global_Sales.sum()
most_pub.sort_values(ascending=False)[:20]
table_countbyPublisher = pd.pivot_table(df[df.Publisher.isin(most_pub.sort_values(ascending=False)[:20].index)],values=['Global_Sales'],index=['Year'],columns=['Publisher'],aggfunc='count',margins=False)
y = table_countbyPublisher.index.astype(int)

plt.figure(figsize=(15,15))
ax = sns.heatmap(table_countbyPublisher['Global_Sales'],linewidths=.5,annot=True,fmt='2.0f',vmin=0,cmap='coolwarm')
ax.set_xlabel(xlabel='Platform', fontsize=20)
ax.set_ylabel(ylabel='Year', fontsize=20)
ax.set_title(label='Count of games by Publisher', fontsize=30)
ax.set_xticklabels(ax.get_xticklabels(), rotation=80)
ax.set_yticklabels(labels = y, fontsize=20)
plt.show()

廠商與類型的熱力圖

In [ ]:
pubGenre = pd.crosstab(df.Publisher,df.Genre)
pubGenre['Total'] = pubGenre.sum(axis=1)
popPublisher = pubGenre[pubGenre['Total']>400].sort_values(by='Total', ascending = False)
needdata = popPublisher.loc[:,:'Strategy']
maxi = needdata.values.max()
mini = needdata.values.min()
popPublisherfinal = popPublisher.append(pd.DataFrame(popPublisher.sum(), columns=['total']).T, ignore_index=False)
sns.set(font_scale=1)
plt.figure(figsize=(15,10))
ax = sns.heatmap(popPublisherfinal, vmin = mini, vmax = maxi, annot=True, fmt="d",cmap='coolwarm')

ax.set_xlabel(xlabel='Genre', fontsize=20)
ax.set_ylabel(ylabel='Publisher', fontsize=20)
ax.set_title(label='Count of Genre by Publisher', fontsize=30)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=14)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=14)
plt.show()

# **各類型遊戲代表作品 - 文字雲**

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
stopwords = set(STOPWORDS)

for x in df.Genre.unique():
    wc = WordCloud(background_color="white", max_words=2000, 
                   stopwords=stopwords, max_font_size=40, random_state=42)
    wc.generate(df.Name[df.Genre == x].to_string())
    plt.figure(figsize=(10,10))
    plt.imshow(wc)
    plt.title(x, fontsize=30)
    plt.axis("off")
    plt.show()

# **使用迴歸建立數值預測模型**

In [ ]:
#plt.close()
#fig, axs = plt.subplots(1, 3, sharey=True)
#df.plot(kind='scatter', x='Genre', y='Global_Sales', ax=axs[0], figsize=(16, 8))
#df.plot(kind='scatter', x='Platform', y='Global_Sales', ax=axs[1])
#df.plot(kind='scatter', x='Publisher', y='Global_Sales', ax=axs[2])
#plt.show()